# Scene Detection in Movies and TV shows
implementation of the paper [Rasheed, et al. “Scene detection in Hollywood movies and tv shows” Proc. of IEEE Computer Society Conference on Computer Vision and Pattern Recognition, vol. 2, pp. 343-348, 2003](https://www.crcv.ucf.edu/papers/Scene-segCVPR2003.pdf)

## Read input data and Color shot detection

In [2]:
# import required module
from pathlib import Path
import cv2
import numpy as np
FILE = 'climate_out'  # 'ftfm_out''news_out'
GROUND_FILE = 'climate_ground.txt'

In [3]:
threshold = 0.9
min_shot_duration = 10 # in frames

In [4]:
# iterate over files in that directory
files = [str(p) for p in Path(FILE).glob('*')]
files.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))
hist_list = []
for frame_path in files:
    frame = cv2.imread(frame_path)
    # Compute color histogram
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([frame], [0, 1, 2], None, [8, 4, 4], [0, 180, 0, 256, 0, 256])
    hist = cv2.normalize(hist, hist, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX).flatten()
    hist_list.append(hist)
    
print(len(hist_list))

1780


In [5]:
# # test filter and sort function
# files_test = [str(p) for p in Path(directory).glob('*')]
# files_test.sort(key=lambda f: int(''.join(filter(str.isdigit, f))))

# howard = list(filter(str.isdigit, "002sfi4"))
# print(howard)

In [6]:
# ## Read video
# cap = cv2.VideoCapture('climate.mp4')
# hist_list = []
# ## make histogram of every frame
# while(cap.isOpened()):
#     ret, frame = cap.read()
#     if ret == True:
#         # Compute color histogram
#         frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
#         hist = cv2.calcHist([frame], [0, 1, 2], None, [8, 4, 4], [0, 180, 0, 256, 0, 256])
#         # print("new\nhist:{}\n".format(hist.shape))
#         hist = cv2.normalize(hist, hist, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX).flatten()
#         # print(hist)
#         # print("end")
#         hist_list.append(hist)
#     else:
#         break
    
# print(len(hist_list))

In [7]:
shot_boundaries = []
for i in range(1, len(hist_list)):
    # Compute histogram difference
    diff = cv2.compareHist(hist_list[i-1], hist_list[i], cv2.HISTCMP_CORREL)
    if diff < threshold:
        shot_boundaries.append(i+1)
print(shot_boundaries)

[93, 157, 232, 314, 355, 460, 691, 697, 886, 900, 919, 1021, 1237, 1401, 1555]


In [8]:
shot_lengths = np.diff(shot_boundaries)
short_shots = np.where(shot_lengths < min_shot_duration)[0]

print(shot_boundaries)

[93, 157, 232, 314, 355, 460, 691, 697, 886, 900, 919, 1021, 1237, 1401, 1555]


In [9]:

for i in reversed(short_shots):
    if i == len(shot_boundaries)-1:
        shot_boundaries = shot_boundaries[:i+1]
    else:
        shot_boundaries = shot_boundaries[:i+1] + shot_boundaries[i+2:]
print(shot_boundaries)

[93, 157, 232, 314, 355, 460, 691, 886, 900, 919, 1021, 1237, 1401, 1555]


## Read ground truth file

In [10]:
ground_file = open(GROUND_FILE, "r")
ground_data = ground_file.read()
ground_list = ground_data.split("\n")
ground_file.close()


In [11]:
ground_list = ground_list[4:]

In [12]:
for i in range(len(ground_list)):
    if '~' in ground_list[i]:
        print(ground_list[i])
        first, second = ground_list[i].split("~")
        ground_list[i] = (first, second)

455~478
542~578
608~644
675~697
774~799
886~887


In [13]:
print(ground_list)

['93',
 '157',
 '232',
 '314',
 '355',
 ('455', '478'),
 ('542', '578'),
 ('608', '644'),
 ('675', '697'),
 ('774', '799'),
 ('886', '887'),
 '1021',
 '1237',
 '1401',
 '1555']

In [14]:
true_positive = 0
false_positive = 0
false_negative = 0
true_negative = 0

for shot in shot_boundaries:
    hit = 0
    for number in ground_list:
        if type(number) is tuple:
            (first ,second ) = number
            if int(first) <= shot <= int(second):
                true_positive += 1
                hit = 1
                break
        else:
            if shot == int(number):
                true_positive += 1
                hit = 1
                break
            
    if hit == 0:
        false_positive += 1

In [15]:
print(true_positive, false_positive)

12 2


In [16]:
for number in ground_list:
    hit = 0
    for shot in shot_boundaries:
        if type(number) is tuple:
            (first ,second) = number
            if int(first) <= shot <= int(second):
                hit = 1
                break
        else:
            if shot == int(number):
                hit = 1
                break
    if hit == 0:
        false_negative += 1

In [18]:
true_negative = len(files) - len(shot_boundaries) - false_negative

In [17]:
print(false_negative, true_negative)

3

In [22]:
precision = true_positive/(true_positive + false_positive)
recall = true_positive/(true_positive + false_negative)
print(precision, recall)

0.8571428571428571 0.8
